In [1]:
# Путь к экземплярам тестов и результам их вычисления
# Директория, где расположены json'ы
default_problem_instances_path = "./data"

In [424]:
import os
import json
import asyncio
import itertools
import aiofiles
import aiofiles.os
import pandas as pd

from collections import defaultdict
from tqdm.notebook import trange, tqdm

In [425]:
# Константы

INF = float("inf")

In [426]:
# nget
# Вернуть значение сложного объекта по указанному пути
def nget(obj, path, defval=None):
    for k in path:
        if obj is None:
            return defval
        elif hasattr(obj, 'get'):
            if k not in obj:
                return defval
            obj = obj.get(k)
        elif hasattr(obj, '__iter__'):
            if type(k) != int:
                return defval
            elif len(obj) - 1 < k:
                return defval
            else:
                obj = obj[k]
        else:
            return defval
    return obj

# Записать значение сложного объекта по указанному пути
def nset(obj, path, val):
    _obj = nget(obj, path[0:-1])
    if _obj:
        _obj[path[-1]] = val
    return val

In [427]:
# _mat3d

# Конвертировать 2d представление задачи в 3d
def _mat3d(mat2d, shape):
    _res3d = []
    _mat2d = []
    shape = shape.copy()
    p = shape.pop(0)
    for row in mat2d:
        _mat2d.append(row)
        if len(_mat2d) == p:
            _res3d.append(_mat2d)
            if shape:
                p = shape.pop(0)
            _mat2d = []
    return _res3d

In [428]:
# _mat2d

# Конвертировать 3d представление задачи в 2d
def _mat2d(mat3d):
    _res2d = []
    for mat in mat3d:
        _res2d.extend(mat)
    return _res2d

In [429]:
def get_schedule_vec(x):
    vec = [0] * len(x[0][0])
    for j, _ in enumerate(x):
        for k, _ in enumerate(x[j]):
            for t, _ in enumerate(x[j][k]):
                if x[j][k][t]:
                    vec[t] = j + 1
    return vec

In [430]:
# get_job_time_slots

# Вернуть время, в которое выполняется каждая работа
def get_job_time_slots(s):
    # s: list = расписание
    job_times = defaultdict(list)
    for t, j in enumerate(s, start=1):
        job_times[j].append(t)
    return job_times

# Проверить прерывает ли работа j2 работу j1
def is_job_intersects(j2, j1, job_times):
    # j1: int = номер прерываемой работы
    # j2: int = номер прерывающей работы
    # job_times: get_job_time_slots()
    if not job_times[j1] or not job_times[j2]:
        return False
    start = job_times[j1][0]
    compl = job_times[j1][-1]
    for t in job_times[j2]:
        if start < t < compl:
            return True
    return False

# Проверить наличие взаимных прерываний
def has_mutual_job_intersection(s, job_times=None):
    # s: list = расписание
    # job_times: get_job_time_slots()
    if not job_times:
        job_times = get_job_time_slots(s)
    for j1, j2 in itertools.combinations(job_times.keys(), 2):
        if (
                is_job_intersects(j2, j1, job_times)
            and is_job_intersects(j1, j2, job_times)
        ):
            return True
    return False

# Проверить выполняется ли работа вовремя: вовремя запускается и вовремя завершается
def is_job_processed_intime(j, params, job_times):
    # j: int = номер работы
    # params: object = параметры работ
    # job_times: get_job_time_slots()
    if not job_times:
        return False
    start = job_times[j][0]
    compl = job_times[j][-1]
    h = j - 1
    if (
            (params["s1"][h] <= start <= params["s2"][h])
        and (params["d1"][h] <= compl <= params["d2"][h])
    ):
        return True
    return False

# Проверить выполняемость вовремя работ. Вернуть список [True, False, ...]
def get_processed_intime_jobs(s, params, job_times=None):
    # s: list = расписание
    # params: object = параметры работ
    # job_times: get_job_time_slots()
    if not job_times:
        job_times = get_job_time_slots(s)
    jobs = sorted(job_times.keys(), reverse=False)
    return [
        is_job_processed_intime(j, params, job_times)
        for j in jobs
    ]

# ----------------------------
# Тест
# ----------------------------
def main():
    test_cases = [
        { "s1": [1, 7, 4], "s2": [ 3,  9,  5], "d1": [3, 10, 7],  "d2": [ 3, 10, 10] },    # -> 1,1,1
        { "s1": [1, 1, 1], "s2": [10, 10, 10], "d1": [1,  1, 1],  "d2": [10, 10, 10] },    # -> 1,1,1
        { "s1": [1, 1, 1], "s2": [10, 10, 10], "d1": [1,  1, 4],  "d2": [10, 10,  7] },    # -> 1,1,0
        { "s1": [2, 1, 1], "s2": [ 3, 10, 10], "d1": [1,  1, 1],  "d2": [10,  8, 10] },    # -> 0,0,1
    ]

    #                1  2  3  4  5  6  7  8  9  10
    test_schedule = [1, 1, 1, 3, 3, 3, 3, 3, 2, 2]
    
    for i, params in enumerate(test_cases, 1):
        print(i, get_processed_intime_jobs(test_schedule, params))
# main()

# ----------------------------
# Тест
# ----------------------------
def main():
    test_cases = [
        [1, 1, 1, 3, 3, 3, 3, 3, 2, 2],    # Без прерываний
        [1, 1, 2, 3, 1, 2, 3, 3, 3, 3],    # Взаимное прерывание двух работ
        [1, 1, 1, 3, 3, 2, 2, 3, 3, 3],    # Одна работа прерывает другую
    ]
    
    for i, s in enumerate(test_cases, 1):
        print(s, has_mutual_job_intersection(s))
# main()

In [431]:
# BaseProblemInstanceLoader
# Загрузчик датасета с диска
class BaseProblemInstanceLoader:
    def __init__(self, path):
        self._path = path
        self._items = None
        self._curr = None
        self._dirs = os.listdir(self._path)
        self._dirs = list(
            filter(
                lambda x: x[0] != "_",
                self._dirs,
            )
        )

    def __iter__(self):
        return self
        
    def __next__(self):
        if (not self._dirs) and (not self._items):
            raise StopIteration

        if not self._items:
            _dirname    = self._dirs.pop(0)
            self._curr  = os.path.join(self._path, _dirname)
            self._items = os.listdir(self._curr)

        _filename = self._items.pop(0)
        _filepath = os.path.join(self._curr, _filename) 

        fd = open(_filepath, "r", encoding="utf-8")
        data = json.load(fd)
        fd.close()

        return data

class PILStd(BaseProblemInstanceLoader):
    def __next__(self):
        instance  = super().__next__()
        return instance

# Вариант без решений -- меньше памяти
class PILVerbose0(PILStd):
    def __next__(self):
        instance = super().__next__()
        for test_name in instance["results"]:
            if "x" in instance["results"][test_name]:
                instance["results"][test_name]["x"] = None
        return instance

In [432]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', '{:.2f}'.format)

In [433]:
# Загрузить задачи с диска.
# Собрать их в таблицу для подсчета статистики

def main():
    global problem_instances_df
    
    _table = []
    
    pil = PILStd(default_problem_instances_path + "/json")
    
    for row in tqdm(pil):
        n = len(nget(row, ["params", "p"], []))
        p = nget(row, ["params", "p", 0])
        x = nget(row, ["results", "osp_lap_bnb_lc_arc", "x"])
        
        schedule_vec = get_schedule_vec(x)
        all_intime   = all(get_processed_intime_jobs(schedule_vec, row["params"]))
        intersec     = has_mutual_job_intersection(schedule_vec)
        utol_correct = nget(row, ["results", "osp_blp", "obj"]) == nget(row, ["results", "osp_lap_bnb_lc_arc_utol", "obj"])
        
        test_types = row["results"].keys()
        test_props = ["obj", "exec_time", "n_tree_nodes", "n_lap_solved"]
        
        _intime   = nget(row, ["results", "all_intime"], -1)
        _intersec = nget(row, ["results", "intersec"], -1)
        
        _table_row = {
            "n": n,
            "p": p,
            "np": n*p,
            "all_intime": -1 if all_intime is None else int(all_intime),
            "intersec": -1 if intersec is None else int(intersec),
            "osp_lap_bnb_lc_arc_utol.correct": int(utol_correct),
        }
        
        for test_type in test_types:
            for prop in test_props:
                _table_row[f"{test_type}.{prop}"] = nget(row, ["results", test_type, prop], -1)
        
        _table.append(_table_row)
        
    problem_instances_df = pd.DataFrame(_table).sort_values(by="np")
    problem_instances_df.fillna(-1, inplace=True)
main()

0it [00:00, ?it/s]

In [434]:
print("Сводная таблица данных")
display(problem_instances_df)

Сводная таблица данных


,n,p,np,all_intime,intersec,osp_lap_bnb_lc_arc_utol.correct,osp_blp.obj,osp_blp.exec_time,osp_blp.n_tree_nodes,osp_blp.n_lap_solved,osp_lap_bnb_1st_arc.obj,osp_lap_bnb_1st_arc.exec_time,osp_lap_bnb_1st_arc.n_tree_nodes,osp_lap_bnb_1st_arc.n_lap_solved,osp_lap_bnb_hc_arc.obj,osp_lap_bnb_hc_arc.exec_time,osp_lap_bnb_hc_arc.n_tree_nodes,osp_lap_bnb_hc_arc.n_lap_solved,osp_lap_bnb_lc_arc.obj,osp_lap_bnb_lc_arc.exec_time,osp_lap_bnb_lc_arc.n_tree_nodes,osp_lap_bnb_lc_arc.n_lap_solved,osp_lap_bnb_lc_arc_utol.obj,osp_lap_bnb_lc_arc_utol.exec_time,osp_lap_bnb_lc_arc_utol.n_tree_nodes,osp_lap_bnb_lc_arc_utol.n_lap_solved,osp_lap_bnb_lo_utol.obj,osp_lap_bnb_lo_utol.exec_time,osp_lap_bnb_lo_utol.n_tree_nodes,osp_lap_bnb_lo_utol.n_lap_solved
0,3,2,6,0,1,1,41,0.02,-1,-1,41.00,0.00,1.00,1.00,41.00,0.00,1.00,1.00,41,0.00,1,1,41,0.00,1,1,41,0.00,1,1
658,3,2,6,0,1,1,40,0.00,-1,-1,40.00,0.00,1.00,1.00,40.00,0.00,1.00,1.00,40,0.00,1,1,40,0.00,1,1,40,0.00,1,1
659,3,2,6,0,1,1,27,0.00,-1,-1,27.00,0.00,1.00,1.00,27.00,0.00,1.00,1.00,27,0.00,1,1,27,0.00,1,1,27,0.00,1,1
660,3,2,6,0,0,1,48,0.02,-1,-1,48.00,0.00,1.00,1.00,48.00,0.00,1.00,1.00,48,0.00,1,1,48,0.00,1,1,48,0.00,1,1
661,3,2,6,0,0,1,150,0.00,-1,-1,150.00,0.00,1.00,1.00,150.00,0.00,1.00,1.00,150,0.00,1,1,150,0.00,1,1,150,0.00,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7073,4,5,20,0,1,1,156,0.00,-1,-1,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,156,15.47,48661,24331,156,6.73,13097,8207,156,16.97,26557,23278
7074,4,5,20,0,1,0,164,0.00,-1,-1,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,164,30.58,81173,40587,172,26.17,57437,35949,164,25.80,45457,39066
7075,4,5,20,0,1,1,135,0.00,-1,-1,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,135,1741.80,5276965,2638483,135,257.94,557661,356606,135,239.45,417165,379480
7065,4,5,20,0,1,1,76,0.00,-1,-1,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,76,62.31,209817,104909,76,25.88,61273,39410,76,93.88,147945,141699


In [435]:
# Средние значения числа решенных ЗН. 1A, LCA, HCA. Мин., макс., медиана. Квантиль 0.05 <= n_lap <= 0.95
def main():
    global exec_test_stats_df
    
    n_list = [3, 4, 5]
    p_list = [2, 3, 4, 5]

    _table = []

    for n in n_list:
        for p in p_list:
            df = problem_instances_df.copy()
            df = df[
                  (df["n"] == n)
                & (df["p"] == p)
            ]

            n_tests = df.shape[0]

            if not n_tests:
                continue

            # df = df[df["osp_lap_bnb_lc_arc.n_lap_solved"] > 1]
            # df_lc = df_hc = df_1st = df

            qcol = "osp_lap_bnb_lc_arc.n_lap_solved"
            q0, q1 = df[qcol].quantile(0.05), df[qcol].quantile(0.95)
            df_lc = df[(q0 <= df[qcol]) & (df[qcol] <= q1)]

            qcol = "osp_lap_bnb_hc_arc.n_lap_solved"
            q0, q1 = df[qcol].quantile(0.05), df[qcol].quantile(0.95)
            df_hc = df[(q0 <= df[qcol]) & (df[qcol] <= q1)]

            qcol = "osp_lap_bnb_1st_arc.n_lap_solved"
            q0, q1 = df[qcol].quantile(0.05), df[qcol].quantile(0.95)
            df_1st = df[(q0 <= df[qcol]) & (df[qcol] <= q1)]

            _row = {
                "n"          : n,
                "p"          : p,
                "np"         : n*p,
                "N"          : n_tests,
                
                "1sta.mean_n_lap": df["osp_lap_bnb_1st_arc.n_lap_solved"].mean(),
                "hca.mean_n_lap" : df["osp_lap_bnb_hc_arc.n_lap_solved"].mean(),
                "lca.mean_n_lap" : df["osp_lap_bnb_lc_arc.n_lap_solved"].mean(),
                
                "q_1sta.mean_n_lap": df_1st["osp_lap_bnb_1st_arc.n_lap_solved"].mean(),
                "q_hca.mean_n_lap" : df_hc["osp_lap_bnb_hc_arc.n_lap_solved"].mean(),
                "q_lca.mean_n_lap" : df_lc["osp_lap_bnb_lc_arc.n_lap_solved"].mean(),
                
                "q_1sta.min_n_lap": df_1st["osp_lap_bnb_1st_arc.n_lap_solved"].min(),
                "q_hca.min_n_lap" : df_hc["osp_lap_bnb_hc_arc.n_lap_solved"].min(),
                "q_lca.min_n_lap" : df_lc["osp_lap_bnb_lc_arc.n_lap_solved"].min(),

                "q_1sta.med_n_lap": df_1st["osp_lap_bnb_1st_arc.n_lap_solved"].median(),
                "q_hca.med_n_lap" : df_hc["osp_lap_bnb_hc_arc.n_lap_solved"].median(),
                "q_lca.med_n_lap" : df_lc["osp_lap_bnb_lc_arc.n_lap_solved"].median(),
                
                "q_1sta.max_n_lap": df_1st["osp_lap_bnb_1st_arc.n_lap_solved"].max(),
                "q_hca.max_n_lap" : df_hc["osp_lap_bnb_hc_arc.n_lap_solved"].max(),
                "q_lca.max_n_lap" : df_lc["osp_lap_bnb_lc_arc.n_lap_solved"].max(),
            }

            _table.append(_row)

        # end for p
    # end for n

    df = pd.DataFrame(_table)
    df = df.sort_values(by=["n","p"], ascending=True)

    print("Средние значения числа решенных ЗН. 1A, LCA, HCA. Мин., макс., медиана")
    display(df)
main()

Средние значения числа решенных ЗН. 1A, LCA, HCA. Мин., макс., медиана


,n,p,np,N,1sta.mean_n_lap,hca.mean_n_lap,lca.mean_n_lap,q_1sta.mean_n_lap,q_hca.mean_n_lap,q_lca.mean_n_lap,q_1sta.min_n_lap,q_hca.min_n_lap,q_lca.min_n_lap,q_1sta.med_n_lap,q_hca.med_n_lap,q_lca.med_n_lap,q_1sta.max_n_lap,q_hca.max_n_lap,q_lca.max_n_lap
0,3,2,6,1000,1.30,1.30,1.31,1.29,1.29,1.29,1.00,1.00,1,1.00,1.00,1.00,3.00,3.00,3
1,3,3,9,1000,12.31,12.85,12.01,10.81,11.33,10.56,1.00,1.00,1,9.00,10.00,9.00,32.00,34.00,32
2,3,4,12,1000,168.95,191.41,156.73,147.54,166.66,140.44,8.00,9.00,8,121.50,135.00,116.00,501.00,557.00,451
3,3,5,15,1000,4429.72,5287.07,3656.72,3283.73,4087.46,2817.68,110.00,141.00,111,1798.50,2466.50,1640.00,16322.00,20269.00,12921
4,4,2,8,1000,1.44,1.44,1.46,1.39,1.39,1.38,1.00,1.00,1,1.00,1.00,1.00,3.00,3.00,3
5,4,3,12,1000,60.57,66.69,56.85,52.09,57.49,49.86,5.00,5.00,5,40.00,44.00,39.00,182.00,207.00,167
6,4,4,16,1000,3986.41,4407.43,3229.53,2760.48,3163.19,2385.87,156.00,167.00,161,1740.00,2118.00,1531.00,12631.00,13384.00,10969
7,4,5,20,100,-1.00,-1.00,307056.17,-1.00,-1.00,163248.70,-1.00,-1.00,636,-1.00,-1.00,78350.00,-1.00,-1.00,1097525
8,5,2,10,1000,-1.00,-1.00,1.55,-1.00,-1.00,1.42,-1.00,-1.00,1,-1.00,-1.00,1.00,-1.00,-1.00,3
9,5,3,15,1000,-1.00,-1.00,289.51,-1.00,-1.00,215.26,-1.00,-1.00,19,-1.00,-1.00,140.00,-1.00,-1.00,1109


In [436]:
# Средние значения числа решенных ЗН. LCA, LCAUT, LUT.

def main():
    global exec_test_stats_df
    
    n_list = [3, 4, 5]
    p_list = [2, 3, 4, 5]

    _table = []

    for n in n_list:
        for p in p_list:
            df = problem_instances_df.copy()
            df = df[
                  (df["n"] == n)
                & (df["p"] == p)
            ]

            n_tests = df.shape[0]

            if not n_tests:
                continue

            _row = {
                "n"          : n,
                "p"          : p,
                "np"         : n*p,
                "N"          : n_tests,
                
                "%lca.n_lap_1": (df["osp_lap_bnb_lc_arc.n_lap_solved"] == 1).mean(),
                "%lca_ut_eq": df["osp_lap_bnb_lc_arc_utol.correct"].mean(),
                
                "lca.mean_n_lap": df["osp_lap_bnb_lc_arc.n_lap_solved"].mean(),
                "lca_ut.mean_n_lap": df["osp_lap_bnb_lc_arc_utol.n_lap_solved"].mean(),
                "lut.mean_n_lap": df["osp_lap_bnb_lo_utol.n_lap_solved"].mean(),
            }

            _table.append(_row)

        # end for p
    # end for n

    df = pd.DataFrame(_table)
    df = df.sort_values(by=["n","p"], ascending=True)

    print("Средние значения числа решенных ЗН")
    display(df)

main()

Средние значения числа решенных ЗН


,n,p,np,N,%lca.n_lap_1,%lca_ut_eq,lca.mean_n_lap,lca_ut.mean_n_lap,lut.mean_n_lap
0,3,2,6,1000,0.85,1.00,1.31,1.28,1.42
1,3,3,9,1000,0.12,0.98,12.01,10.03,14.94
2,3,4,12,1000,0.03,0.97,156.73,90.72,144.37
3,3,5,15,1000,0.01,0.94,3656.72,1341.63,2060.93
4,4,2,8,1000,0.80,1.00,1.46,1.45,1.64
5,4,3,12,1000,0.02,0.97,56.85,43.48,72.37
6,4,4,16,1000,0.00,0.96,3229.53,1459.21,2495.72
7,4,5,20,100,0.01,0.93,307056.17,98197.81,157814.02
8,5,2,10,1000,0.76,1.00,1.55,1.54,1.79
9,5,3,15,1000,0.01,0.96,289.51,215.15,369.79


In [437]:
# Средние значения числа решенных ЗН. Квантиль 0.05 <= n_lap <= 0.95

def main():
    global exec_test_stats_df
    
    n_list = [3, 4, 5]
    p_list = [2, 3, 4, 5]

    _table = []

    for n in n_list:
        for p in p_list:
            df = problem_instances_df.copy()
            df = df[
                  (df["n"] == n)
                & (df["p"] == p)
            ]

            n_tests = df.shape[0]

            if not n_tests:
                continue

            qcol = "osp_lap_bnb_lc_arc.n_lap_solved"
            q0 = df[qcol].quantile(0.05)
            q1 = df[qcol].quantile(0.95)
            df_lc = df[(q0 <= df[qcol]) & (df[qcol] <= q1)]

            qcol = "osp_lap_bnb_lc_arc_utol.n_lap_solved"
            q0 = df[qcol].quantile(0.05)
            q1 = df[qcol].quantile(0.95)
            df_lc_utol = df[(q0 <= df[qcol]) & (df[qcol] <= q1)]

            qcol = "osp_lap_bnb_lo_utol.n_lap_solved"
            q0 = df[qcol].quantile(0.05)
            q1 = df[qcol].quantile(0.95)
            df_lo_utol = df[(q0 <= df[qcol]) & (df[qcol] <= q1)]
            
            _row = {
                "n"          : n,
                "p"          : p,
                "np"         : n*p,
                "N"          : n_tests,
                
                "%q_lca.n_lap_1": (df_lc["osp_lap_bnb_lc_arc.n_lap_solved"] == 1).mean(),
                "%q_lca_ut.eq": df_lc_utol["osp_lap_bnb_lc_arc_utol.correct"].mean(),
                
                "q_lca.mean_n_lap": df_lc["osp_lap_bnb_lc_arc.n_lap_solved"].mean(),
                "q_lca_ut.mean_n_lap": df_lc_utol["osp_lap_bnb_lc_arc_utol.n_lap_solved"].mean(),
                "q_lut.mean_n_lap": df_lo_utol["osp_lap_bnb_lo_utol.n_lap_solved"].mean(),
            }

            _table.append(_row)

        # end for p
    # end for n

    df = pd.DataFrame(_table)
    df = df.sort_values(by=["n","p"], ascending=True)

    print("Средние значения числа решенных ЗН. Квантиль 0.05 <= n_lap <= 0.95")
    display(df)

main()

Средние значения числа решенных ЗН. Квантиль 0.05 <= n_lap <= 0.95


,n,p,np,N,%q_lca.n_lap_1,%q_lca_ut.eq,q_lca.mean_n_lap,q_lca_ut.mean_n_lap,q_lut.mean_n_lap
0,3,2,6,1000,0.86,1.00,1.29,1.27,1.42
1,3,3,9,1000,0.12,0.99,10.56,9.00,13.14
2,3,4,12,1000,0.00,0.97,140.44,77.39,122.78
3,3,5,15,1000,0.00,0.94,2817.68,979.90,1501.02
4,4,2,8,1000,0.81,1.00,1.38,1.38,1.58
5,4,3,12,1000,0.00,0.97,49.86,36.67,61.47
6,4,4,16,1000,0.00,0.95,2385.87,1129.42,1953.00
7,4,5,20,100,0.00,0.94,163248.70,46212.67,81179.16
8,5,2,10,1000,0.79,1.00,1.42,1.42,1.64
9,5,3,15,1000,0.00,0.96,215.26,153.88,273.89


In [438]:
# Количество решенных ЗН: мин., макс., медиана
def main():
    global exec_test_stats_df
    
    n_list = [3, 4, 5]
    p_list = [2, 3, 4, 5]

    _table = []

    for n in n_list:
        for p in p_list:
            df = problem_instances_df.copy()
            df = df[
                  (df["n"] == n)
                & (df["p"] == p)
            ]

            n_tests = df.shape[0]

            if not n_tests:
                continue

            _row = {
                "n"          : n,
                "p"          : p,
                "np"         : n*p,

                "N"          : n_tests,

                # "%lca.n_lap_1": (df["osp_lap_bnb_lc_arc.n_lap_solved"] == 1).mean(),
                # "%lca_ut_eq": df["osp_lap_bnb_lc_arc_utol.correct"].mean(),

                "n_lca.n_lap_1": (df["osp_lap_bnb_lc_arc.n_lap_solved"] == 1).sum(),
                "n_lca_ut_eq": df["osp_lap_bnb_lc_arc_utol.correct"].sum(),

                "lca.min_n_lap": df["osp_lap_bnb_lc_arc.n_lap_solved"].min(),
                "lca_ut.min_n_lap": df["osp_lap_bnb_lc_arc_utol.n_lap_solved"].min(),
                "lut.min_n_lap": df["osp_lap_bnb_lo_utol.n_lap_solved"].min(),

                "lca.med_n_lap": df["osp_lap_bnb_lc_arc.n_lap_solved"].median(),
                "lca_ut.med_n_lap": df["osp_lap_bnb_lc_arc_utol.n_lap_solved"].median(),
                "lut.med_n_lap": df["osp_lap_bnb_lo_utol.n_lap_solved"].median(),
                
                "lca.max_n_lap": df["osp_lap_bnb_lc_arc.n_lap_solved"].max(),
                "lca_ut.max_n_lap": df["osp_lap_bnb_lc_arc_utol.n_lap_solved"].max(),
                "lut.max_n_lap": df["osp_lap_bnb_lo_utol.n_lap_solved"].max(),
            }

            _table.append(_row)

        # end for p
    # end for n

    df = pd.DataFrame(_table)
    df = df.sort_values(by=["n","p"], ascending=True)

    print("Количество решенных ЗН: мин., макс., медиана")
    display(df)

main()

Количество решенных ЗН: мин., макс., медиана


,n,p,np,N,n_lca.n_lap_1,n_lca_ut_eq,lca.min_n_lap,lca_ut.min_n_lap,lut.min_n_lap,lca.med_n_lap,lca_ut.med_n_lap,lut.med_n_lap,lca.max_n_lap,lca_ut.max_n_lap,lut.max_n_lap
0,3,2,6,1000,854,1000,1,1,1,1.00,1.00,1.00,8,7,4
1,3,3,9,1000,115,985,1,1,1,10.00,8.00,13.00,81,62,106
2,3,4,12,1000,25,970,1,1,1,116.00,60.00,93.50,1198,751,1752
3,3,5,15,1000,11,945,1,1,1,1640.00,447.00,731.50,55744,24679,37110
4,4,2,8,1000,796,1000,1,1,1,1.00,1.00,1.00,9,9,7
5,4,3,12,1000,22,972,1,1,1,39.00,28.00,46.50,473,375,616
6,4,4,16,1000,4,956,1,1,1,1534.50,644.50,1053.50,102339,22068,34471
7,4,5,20,100,1,93,1,1,1,78350.00,14514.50,29525.50,4851259,4228197,5774336
8,5,2,10,1000,764,1000,1,1,1,1.00,1.00,1.00,9,9,10
9,5,3,15,1000,5,960,1,1,1,140.00,92.00,155.00,9331,7270,9370


In [439]:
# Количество решенных ЗН: мин., макс., медиана. Квантиль 0.05 <= n_lap <= 0.95
def main():
    n_list = [3, 4, 5]
    p_list = [2, 3, 4, 5]
    _table = []

    for n in n_list:
        for p in p_list:
            df = problem_instances_df.copy()
            df = df[(df["n"] == n) & (df["p"] == p)]

            n_tests = df.shape[0]

            if not n_tests:
                continue

            qcol = "osp_lap_bnb_lc_arc.n_lap_solved"
            q0 = df[qcol].quantile(0.05)
            q1 = df[qcol].quantile(0.95)
            df_lc = df[(q0 <= df[qcol]) & (df[qcol] <= q1)]

            qcol = "osp_lap_bnb_lc_arc_utol.n_lap_solved"
            q0 = df[qcol].quantile(0.05)
            q1 = df[qcol].quantile(0.95)
            df_lc_utol = df[(q0 <= df[qcol]) & (df[qcol] <= q1)]

            qcol = "osp_lap_bnb_lo_utol.n_lap_solved"
            q0 = df[qcol].quantile(0.05)
            q1 = df[qcol].quantile(0.95)
            df_lo_utol = df[(q0 <= df[qcol]) & (df[qcol] <= q1)]

            _row = {
                "n"          : n,
                "p"          : p,
                "np"         : n*p,

                "N_lca"      : df_lc.shape[0],
                "N_lca_ut"   : df_lc_utol.shape[0],
                "N_lut"      : df_lo_utol.shape[0],

                "q_n_lca.n_lap_1": (df_lc["osp_lap_bnb_lc_arc.n_lap_solved"] == 1).sum(),
                "q_n_lca_ut_eq": df_lc_utol["osp_lap_bnb_lc_arc_utol.correct"].sum(),

                "q_lca.min_n_lap": df_lc["osp_lap_bnb_lc_arc.n_lap_solved"].min(),
                "q_lca_ut.min_n_lap": df_lc_utol["osp_lap_bnb_lc_arc_utol.n_lap_solved"].min(),
                "q_lut.min_n_lap": df_lo_utol["osp_lap_bnb_lo_utol.n_lap_solved"].min(),

                "q_lca.med_n_lap": df_lc["osp_lap_bnb_lc_arc.n_lap_solved"].median(),
                "q_lca_ut.med_n_lap": df_lc_utol["osp_lap_bnb_lc_arc_utol.n_lap_solved"].median(),
                "q_lut.med_n_lap": df_lo_utol["osp_lap_bnb_lo_utol.n_lap_solved"].median(),
                
                "q_lca.max_n_lap": df_lc["osp_lap_bnb_lc_arc.n_lap_solved"].max(),
                "q_lca_ut.max_n_lap": df_lc_utol["osp_lap_bnb_lc_arc_utol.n_lap_solved"].max(),
                "q_lut.max_n_lap": df_lo_utol["osp_lap_bnb_lo_utol.n_lap_solved"].max(),
            }

            _table.append(_row)
        # end for p
    # end for n

    df = pd.DataFrame(_table)
    df = df.sort_values(by=["n","p"], ascending=True)

    print("Количество решенных ЗН: мин., макс., медиана. Квантиль 0.05 <= n_lap <= 0.95")
    display(df)
main()

Количество решенных ЗН: мин., макс., медиана. Квантиль 0.05 <= n_lap <= 0.95


,n,p,np,N_lca,N_lca_ut,N_lut,q_n_lca.n_lap_1,q_n_lca_ut_eq,q_lca.min_n_lap,q_lca_ut.min_n_lap,q_lut.min_n_lap,q_lca.med_n_lap,q_lca_ut.med_n_lap,q_lut.med_n_lap,q_lca.max_n_lap,q_lca_ut.max_n_lap,q_lut.max_n_lap
0,3,2,6,996,998,1000,854,998,1,1,1,1.00,1.00,1.00,3,3,4
1,3,3,9,950,958,952,115,944,1,1,1,9.00,8.00,12.00,32,27,40
2,3,4,12,900,925,925,0,899,8,3,4,116.00,57.00,88.00,451,279,437
3,3,5,15,900,901,910,0,850,111,13,16,1640.00,446.00,720.50,12921,5702,8006
4,4,2,8,981,982,989,796,982,1,1,1,1.00,1.00,1.00,3,3,4
5,4,3,12,904,928,928,0,903,5,3,4,39.00,27.00,44.00,167,128,218
6,4,4,16,901,901,900,0,860,161,15,22,1531.00,640.00,1053.50,10969,5389,9057
7,4,5,20,90,90,90,0,85,636,87,130,78350.00,14514.50,29525.50,1097525,334522,528130
8,5,2,10,969,970,973,764,970,1,1,1,1.00,1.00,1.00,3,3,4
9,5,3,15,900,905,908,0,868,19,9,13,140.00,91.00,154.00,1109,912,1523


In [440]:
# Процессорное время: медиана, мин., макс.

def main():
    global exec_test_stats_df
    
    n_list = [3, 4, 5]
    p_list = [2, 3, 4, 5]

    _table = []

    for n in n_list:
        for p in p_list:
            df = problem_instances_df.copy()
            df = df[
                  (df["n"] == n)
                & (df["p"] == p)
            ]

            n_tests = df.shape[0]

            if not n_tests:
                continue

            _row = {
                "n"          : n,
                "p"          : p,

                "N"          : n_tests,

                "lca.min_t": df["osp_lap_bnb_lc_arc.exec_time"].min(),
                "lca_ut.min_t": df["osp_lap_bnb_lc_arc_utol.exec_time"].min(),
                "lut.min_t": df["osp_lap_bnb_lo_utol.exec_time"].min(),

                "lca.med_t": df["osp_lap_bnb_lc_arc.exec_time"].median(),
                "lca_ut.med_t": df["osp_lap_bnb_lc_arc_utol.exec_time"].median(),
                "lut.med_t": df["osp_lap_bnb_lo_utol.exec_time"].median(),
                
                "lca.max_t": df["osp_lap_bnb_lc_arc.exec_time"].max(),
                "lca_ut.max_t": df["osp_lap_bnb_lc_arc_utol.exec_time"].max(),
                "lut.max_t": df["osp_lap_bnb_lo_utol.exec_time"].max(),
            }

            _table.append(_row)

        # end for p
    # end for n

    df = pd.DataFrame(_table)
    df = df.sort_values(by=["n","p"], ascending=True)

    print("Процессорное время: медиана, мин., макс.")
    display(df)

main()

Процессорное время: медиана, мин., макс.


,n,p,N,lca.min_t,lca_ut.min_t,lut.min_t,lca.med_t,lca_ut.med_t,lut.med_t,lca.max_t,lca_ut.max_t,lut.max_t
0,3,2,1000,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.02
1,3,3,1000,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.02
2,3,4,1000,0.00,0.00,0.00,0.03,0.02,0.02,0.30,0.34,0.38
3,3,5,1000,0.00,0.00,0.00,0.70,0.22,0.30,23.16,13.42,19.17
4,4,2,1000,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.02
5,4,3,1000,0.00,0.00,0.00,0.02,0.02,0.02,0.14,0.11,0.23
6,4,4,1000,0.00,0.00,0.00,0.67,0.33,0.47,45.48,13.91,21.59
7,4,5,100,0.00,0.00,0.00,61.95,11.29,19.15,4113.38,3706.88,4562.64
8,5,2,1000,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.02
9,5,3,1000,0.00,0.00,0.00,0.05,0.03,0.05,3.48,3.50,3.36


In [441]:
# Процессорное время: медиана, мин., макс. Квантиль 0.05 <= n_lap <= 0.95

def main():
    n_list = [3, 4, 5]
    p_list = [2, 3, 4, 5]
    _table = []

    for n in n_list:
        for p in p_list:
            df = problem_instances_df.copy()
            df = df[(df["n"] == n) & (df["p"] == p)]

            n_tests = df.shape[0]

            if not n_tests:
                continue

            qcol = "osp_lap_bnb_lc_arc.exec_time"
            q0 = df[qcol].quantile(0.05)
            q1 = df[qcol].quantile(0.95)
            df_lc = df[(q0 <= df[qcol]) & (df[qcol] <= q1)]

            qcol = "osp_lap_bnb_lc_arc_utol.exec_time"
            q0 = df[qcol].quantile(0.05)
            q1 = df[qcol].quantile(0.95)
            df_lc_utol = df[(q0 <= df[qcol]) & (df[qcol] <= q1)]

            qcol = "osp_lap_bnb_lo_utol.exec_time"
            q0 = df[qcol].quantile(0.05)
            q1 = df[qcol].quantile(0.95)
            df_lo_utol = df[(q0 <= df[qcol]) & (df[qcol] <= q1)]

            _row = {
                "n"          : n,
                "p"          : p,
                
                "N"          : df_lc.shape[0],

                "q_lca.min_t": df_lc["osp_lap_bnb_lc_arc.exec_time"].min(),
                "q_lca_ut.min_t": df_lc_utol["osp_lap_bnb_lc_arc_utol.exec_time"].min(),
                "q_lut.min_t": df_lo_utol["osp_lap_bnb_lo_utol.exec_time"].min(),

                "q_lca.med_t": df_lc["osp_lap_bnb_lc_arc.exec_time"].median(),
                "q_lca_ut.med_t": df_lc_utol["osp_lap_bnb_lc_arc_utol.exec_time"].median(),
                "q_lut.med_t": df_lo_utol["osp_lap_bnb_lo_utol.exec_time"].median(),
                
                "q_lca.max_t": df_lc["osp_lap_bnb_lc_arc.exec_time"].max(),
                "q_lca_ut.max_t": df_lc_utol["osp_lap_bnb_lc_arc_utol.exec_time"].max(),
                "q_lut.max_t": df_lo_utol["osp_lap_bnb_lo_utol.exec_time"].max(),
            }

            _table.append(_row)
        # end for p
    # end for n

    df = pd.DataFrame(_table)
    df = df.sort_values(by=["n","p"], ascending=True)

    print("Процессорное время: медиана, мин., макс. Квантиль 0.05 <= n_lap <= 0.95")
    display(df)
main()

Процессорное время: медиана, мин., макс. Квантиль 0.05 <= n_lap <= 0.95


,n,p,N,q_lca.min_t,q_lca_ut.min_t,q_lut.min_t,q_lca.med_t,q_lca_ut.med_t,q_lut.med_t,q_lca.max_t,q_lca_ut.max_t,q_lut.max_t
0,3,2,997,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,3,3,1000,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.02
2,3,4,962,0.00,0.00,0.00,0.03,0.02,0.02,0.12,0.08,0.11
3,3,5,911,0.05,0.00,0.00,0.69,0.19,0.27,5.64,2.89,3.70
4,4,2,998,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,4,3,971,0.00,0.00,0.00,0.02,0.00,0.02,0.05,0.05,0.05
6,4,4,910,0.06,0.00,0.00,0.66,0.28,0.41,4.84,3.27,5.02
7,4,5,90,0.36,0.05,0.08,61.95,11.12,19.15,890.69,257.94,372.64
8,5,2,985,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,5,3,952,0.00,0.00,0.00,0.05,0.03,0.05,0.44,0.38,0.52


In [442]:
# Взаимопрерывание и успеваемость: абсолютное значение, процентное отношение

def main():
    n_list = [3, 4, 5]
    p_list = [2, 3, 4, 5]
    _table = []
    for n in n_list:
        for p in p_list:
            df = problem_instances_df.copy()
            df = df[(df["n"] == n) & (df["p"] == p)]
            
            n_tests = df.shape[0]
            
            if not n_tests:
                continue
                
            _row = {
                "n"          : n,
                "p"          : p,
                "np"         : n*p,

                "N"          : n_tests,

                "n_intrsc" : df["intersec"].sum(),
                "%_intrsc" : df["intersec"].sum() / n_tests,
                
                "n_intm"   : df["all_intime"].sum(),
                "%_intm"   : df["all_intime"].sum() / n_tests,
            }

            _table.append(_row)
        # end for p
    # end for n

    df = pd.DataFrame(_table)
    df = df.sort_values(by=["n","p"], ascending=True)

    print("Взаимопрерывание и успеваемость: абсолютное значение, процентное отношение")
    display(df)

main()

Взаимопрерывание и успеваемость: абсолютное значение, процентное отношение


,n,p,np,N,n_intrsc,%_intrsc,n_intm,%_intm
0,3,2,6,1000,569,0.57,18,0.02
1,3,3,9,1000,807,0.81,34,0.03
2,3,4,12,1000,829,0.83,22,0.02
3,3,5,15,1000,868,0.87,24,0.02
4,4,2,8,1000,772,0.77,4,0.00
5,4,3,12,1000,916,0.92,17,0.02
6,4,4,16,1000,947,0.95,20,0.02
7,4,5,20,100,97,0.97,2,0.02
8,5,2,10,1000,834,0.83,2,0.00
9,5,3,15,1000,968,0.97,10,0.01
